In [1]:
import json
import csv 
import pandas as pd

from rdflib import Graph, Namespace, URIRef, BNode, Literal
from rdflib.namespace import RDF, FOAF, XSD

In [2]:
data_business = []
csvfile = []
cuisineList = []
existdbList = []
specialList = []

In [3]:
with open('yelp_academic_dataset_business.json', encoding="utf8") as f:
    for line in f:
        data_business.append(json.loads(line))
df = pd.read_csv('csvData.csv')

with open('country_cusine_filtered_list.csv', newline='') as inputfile:
    for row in csv.reader(inputfile):
        cuisineList.append(row[0])
        
with open('category_exist_in_db_List.csv', newline='') as inputfile:
    for row in csv.reader(inputfile):
        existdbList.append(row[0])
        
with open('distinct_with_specialchar.csv', newline='') as inputfile:
    for row in csv.reader(inputfile):
        specialList.append(row[0])

In [4]:
exist1 = pd.read_csv('category_exist_in_db_List.csv',header=None)
exist2 = pd.read_csv('category_exist_in_db_List_v2.csv', header=None)
exist3 = pd.read_csv('category_exist_in_db_List_v3.csv', header=None)

frames = [exist1, exist2, exist3]
result = pd.concat(frames)
result = result.reset_index(drop=True)
#exist_v = exist2.add(exist3, fill_value=0)

display(result)

,0,1
0,Doctors,NaN
1,Acupuncture,NaN
2,Nutritionists,NaN
3,Notaries,NaN
4,Shopping,NaN
...,...,...
770,Gerontologists,Gerontologist
771,Otologists,Otologist
772,Parklets,Parklet
773,Neurotologists,Neurotologist


In [5]:
def string_seperate(string):
    n = 0
    cate_list = []
    a_list = string.split(", ")
    for element in a_list:
        cate_list.append(element)
    n = n +1
    return cate_list

def code_to_state(code):
    n = 0
    for index, row in df.iterrows():
        if str(code) == str(df.loc[n,'Code']):
            return str(df.loc[n,'State'])
        #str(df.loc[n,])
        n = n + 1
        

def change_format_to_db(catagory):
    #this function takes a string in catagory from yelp
    #and gives the correct format to put in dbpedia
    
    #Everthing in string to lowercase
    newcatagory = catagory.lower()
    
    #capitalize first letter 
    newcatagory = catagory.capitalize()
    
    #replace empty with _
    newcatagory = catagory.replace(" ", "_")
    
    return newcatagory

In [6]:
schema = Namespace("https://schema.org/")
yelp_business = Namespace("https://www.yelp.com/biz/")
db_resource = Namespace("http://dbpedia.org/resource/")
g_maps = Namespace("http://geonames.org/")

g = Graph()

In [7]:
def triple_maker_name():
    #for each name that exist for a business, add a triple to the graph
    n = 0
    for i in data_business:
        if len(data_business[n]['name']) > 0 and data_business[n]['name'] != None:
            #append to graph
            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.legalName, Literal(data_business[n]['name'])))
        n = n + 1
    print('name done')

In [8]:
def triple_maker_address():
    #for each address that exist for a business, add a triple to the graph
    n = 0
    for i in data_business:
        #append to graph
        if len(data_business[n]['address']) > 0:
            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.streetAddress, Literal(data_business[n]['address'])))
        n = n + 1
    print('address done')

In [9]:
def triple_maker_state():
    n = 0
    for i in data_business:
        #append to graph
        if len(data_business[n]['state']) > 0:
            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.addressRegion, URIRef(db_resource + str(code_to_state(data_business[n]['state'])))))
        n = n + 1
        
    print('state done')

In [10]:
def triple_maker_postal():
    n = 0
    for i in data_business:
        #append to graph
        if len(data_business[n]['postal_code']) > 0:
            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.postalCode, Literal(data_business[n]['postal_code'])))
        n = n + 1
        
    print('postal done')

In [11]:
def triple_maker_latlong():
    n = 0
    for i in data_business:
        #append to graph
        if data_business[n]['latitude'] != None and len(str(data_business[n]['longitude'])) != None :
            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.geo, URIRef(g_maps + str(data_business[n]['latitude']))))
            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.geo, URIRef(g_maps + str(data_business[n]['longitude']))))
        n = n + 1
    print('lat+long done')

In [12]:
def triple_maker_stars():
    n = 0
    for i in data_business:
        #append to graph
        if len(str(data_business[n]['stars'])) > 0:
            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.starRating, Literal(data_business[n]['stars'])))
        n = n + 1
    print('stars done')

In [13]:
def triple_maker_cg_cuisine():
    n = 0
    for i in data_business:
        if data_business[n]['categories'] != None:
            #split the string in categories to a list, create list as catagory_list
            catagory_list = string_seperate(data_business[n]['categories'])
                
            #for every item in catagory_list, compare it with the cuisineList
            for item in range(len(cuisineList)):
                for element in range(len(catagory_list)):
                    #the item in category exist, add it to the graph
                    if catagory_list[element] == cuisineList[item]:
                        #cause the cuisines exist as country+_cuisine in dbpedia
                        #so every country get added a _cuisine after
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.category, URIRef(db_resource + str(cuisineList[element])+ str('_cuisine'))))
        n = n + 1
    print('category for cuisine done')  

In [14]:
def triple_maker_cg_in_db_exist():
                  
    n = n + 1
    for i in data_business:
        if data_business[n]['categories'] != None:
            #split the string in categories to a list, create list as catagory_list
            catagory_list = string_seperate(data_business[n]['categories'])
                
            #for every item in catagory_list, compare it with a elemenent that exist in dbpedia
            item1 = 0
            for item in range(len(catagory_list)):
                element1 = 0
                for element in range(len(existdbList)):
                    #the item in category exist, add it to the graph
                    if str(catagory_list[item]) == str(existdbList[element]):
                        #the format in dbpedia is first letter is upper case
                        #and the rest of the letters are lowercase
                        #so every string goes through the function change_format_to_db
                        #in order to be the correct format
                        
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.category, URIRef(db_resource + change_format_to_db(existdbList[element]))))
    print('category that exist in db done')

In [15]:
g2 = Graph()

def triple_maker_cg_in_db_exist_2():
    category_exist = 0    
    category_not_exist = 0  
    
    for n in range(len(data_business)):
        if data_business[n]['categories'] != None:
            #split the string in categories to a list, create list as catagory_list
            catagory_list = string_seperate(data_business[n]['categories'])
            #for every item in catagory_list, compare it with the dbpedia
            item1 = 0
            for item in range(len(catagory_list)):
                element1 = 0
                for index, row in df.iterrows():
                    #the item in category exist, add it to the graph
                    if str(catagory_list[item]) == str(row[0]):
                        
                        #the format in dbpedia is first letter is upper case
                        #and the rest of the letters are lowercase
                        #so every string goes through the function change_format_to_db
                        #in order to be the correct format
                        #g2.add((URIRef(yelp_business + data_business[n]['business_id']), schema.category, URIRef(db_resource + change_format_to_db(existdbList[element]))))
                        category_exist = category_exist + 1
                    else:
                        category_not_exist = category_not_exist + 1
    print('category_2 that exist in db done')

In [16]:
def triple_maker_att_RestaurantsTakeOut():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('RestaurantsTakeOut'):
                    if (data_business[n]['attributes']['RestaurantsTakeOut']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.category, URIRef(db_resource + 'Take-out')))

In [17]:
def triple_maker_att_Smoking():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('Smoking'):
                    if (data_business[n]['attributes']['Smoking']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.smokingAllowed, Literal('true')))

In [18]:
def triple_maker_att_BusinessAcceptsCreditCards():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('BusinessAcceptsCreditCards'):
                    if (data_business[n]['attributes']['BusinessAcceptsCreditCards']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.paymentAccepted, Literal('Credit Card')))

In [19]:
def triple_maker_att_BikeParking():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('BikeParking'):
                    if (data_business[n]['attributes']['BikeParking']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.ParkingFacility, URIRef(db_resource + 'Bicycle')))

In [20]:
def triple_maker_att_RestaurantsPriceRange2():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('RestaurantsPriceRange2'):
                    if (data_business[n]['attributes']['RestaurantsPriceRange2']) == '1':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.priceRange, Literal('1')))
                    if (data_business[n]['attributes']['RestaurantsPriceRange2']) == '1':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.priceRange, Literal('2')))
                    if (data_business[n]['attributes']['RestaurantsPriceRange2']) == '1':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.priceRange, Literal('3')))
                    if (data_business[n]['attributes']['RestaurantsPriceRange2']) == '1':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.priceRange, Literal('4')))

In [21]:
def triple_maker_att_BikeParking():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('BikeParking'):
                    if (data_business[n]['attributes']['BikeParking']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.ParkingFacility, URIRef(db_resource + 'Bicycle')))

In [22]:
def triple_maker_att_CoatCheck():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('CoatCheck'):
                    if (data_business[n]['attributes']['CoatCheck']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'Wardrobe')))

In [23]:
def triple_maker_att_RestaurantsDelivery():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('RestaurantsDelivery'):
                    if (data_business[n]['attributes']['RestaurantsDelivery']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.DeliveryMethod, URIRef(db_resource + 'Delivery_service')))

In [24]:
def triple_maker_att_Caters():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('Caters'):
                    if (data_business[n]['attributes']['Caters']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.FoodEstablishment, URIRef(db_resource + 'Catering')))

In [25]:
def triple_maker_att_WiFi():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('WiFi'):
                    if (data_business[n]['attributes']['WiFi']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'Wi-Fi')))

In [26]:
def triple_maker_att_WheelchairAccessible():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('WheelchairAccessible'):
                    if (data_business[n]['attributes']['WheelchairAccessible']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.accessibilityFeature, URIRef(db_resource + 'Wheelchair')))

In [27]:
def triple_maker_att_HappyHour():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('HappyHour'):
                    if (data_business[n]['attributes']['HappyHour']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'Happy_Hour')))

In [28]:
def triple_maker_att_OutdoorSeating():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('OutdoorSeating'):
                    if (data_business[n]['attributes']['OutdoorSeating']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.containsPlace, URIRef(db_resource + 'Outdoor_dining')))

In [29]:
def triple_maker_att_HasTV():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('HasTV'):
                    if (data_business[n]['attributes']['HasTV']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'Television')))

In [30]:
def triple_maker_att_RestaurantsReservations():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('RestaurantsReservations'):
                    if (data_business[n]['attributes']['RestaurantsReservations']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.acceptsReservations, URIRef(db_resource + 'RestaurantsReservations')))

In [31]:
def triple_maker_att_DogsAllowed():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('petsAllowed'):
                    if (data_business[n]['attributes']['petsAllowed']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.acceptsReservations, Literal('true')))

In [32]:
def triple_maker_att_Alcohol():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('Alcohol'):
                    if (data_business[n]['attributes']['Alcohol']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.hasMenuItem, URIRef(db_resource + 'Alcoholic_drink')))

In [33]:
def triple_maker_att_GoodForKids():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('GoodForKids'):
                    if (data_business[n]['attributes']['GoodForKids']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'Family-friendly')))

In [34]:
def triple_maker_att_RestaurantsTableService():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('RestaurantsTableService'):
                    if (data_business[n]['attributes']['RestaurantsTableService']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'Waiting_staff')))

In [35]:
def triple_maker_att_DriveThru():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('DriveThru'):
                    if (data_business[n]['attributes']['DriveThru']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'Drive-through')))

In [36]:
def triple_maker_att_NoiseLevel():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('NoiseLevel'):
                    if (data_business[n]['attributes']['NoiseLevel']) == "u'average'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.Level, Literal('under average')))
                    if (data_business[n]['attributes']['NoiseLevel']) == "u'quiet'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.Level, Literal('under quiet')))
                    if (data_business[n]['attributes']['NoiseLevel']) == "'average'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.Level, Literal('average')))

In [37]:
def triple_maker_att_BusinessAcceptsBitcoin():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('BusinessAcceptsBitcoin'):
                    if (data_business[n]['attributes']['BusinessAcceptsBitcoin']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.paymentAccepted, Literal('Cryptocurrency')))

In [38]:
def triple_maker_att_Music():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('Music'):
                    if (data_business[n]['attributes']['Music']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'Music')))

In [39]:
def triple_maker_att_GoodForDancing():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('GoodForDancing'):
                    if (data_business[n]['attributes']['GoodForDancing']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'Dance')))

In [40]:
def triple_maker_att_AcceptsInsurance():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('AcceptsInsurance'):
                    if (data_business[n]['attributes']['AcceptsInsurance']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.paymentAccepted, Literal('Insurance')))

In [41]:
def triple_maker_att_BYOB():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('BYOB'):
                    if (data_business[n]['attributes']['BYOB']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'BYOB')))

In [42]:
def triple_maker_att_corkage():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('corkage'):
                    if (data_business[n]['attributes']['corkage']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'BYOB')))

In [43]:
def triple_maker_att_BYOBCorkage():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('BYOBCorkage'):
                    if (data_business[n]['attributes']['BYOBCorkage']) == 'yes_free':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'BYOB')))
                    if (data_business[n]['attributes']['BYOBCorkage']) == 'yes_free':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'BYOB')))

In [44]:
byob_list =[]
for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('BYOBCorkage'):
                    if (data_business[n]['attributes']['BYOBCorkage']) not in byob_list:
                        byob_list.append(data_business[n]['attributes']['BYOBCorkage'])

In [45]:
lol = 10

print(type(data_business[lol]['attributes']))
print(data_business[lol]['attributes'])
print(type(string_seperate(data_business[lol]['attributes']['BusinessParking'])))
print(string_seperate(data_business[lol]['attributes']['BusinessParking']))

<class 'dict'>
{'RestaurantsPriceRange2': '2', 'BikeParking': 'True', 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}"}
<class 'list'>
["{'garage': False", "'street': False", "'validated': False", "'lot': False", "'valet': False}"]


In [46]:
'''
triple_maker_name()
triple_maker_address()
triple_maker_state()
triple_maker_postal()
triple_maker_latlong()
triple_maker_stars()
triple_maker_cg_cuisine()
triple_maker_cg_in_db_exist()
'''

'\ntriple_maker_name()\ntriple_maker_address()\ntriple_maker_state()\ntriple_maker_postal()\ntriple_maker_latlong()\ntriple_maker_stars()\ntriple_maker_cg_cuisine()\ntriple_maker_cg_in_db_exist()\n'

In [47]:
#g.serialize(destination="Business_v2.ttl")

In [48]:
#triple_maker_cg_in_db_exist_2()

#print(category_exist)
#print(category_not_exist)